#Getting data from drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/NLP_Lab/raw_data/train.csv')

In [ ]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


# Seperating the comments from labels

In [ ]:
comment = df['comment_text']
print(comment.head())

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
Name: comment_text, dtype: object


In [ ]:
label = df[['toxic', 'severe_toxic' , 'obscene' , 'threat' , 'insult' , 'identity_hate']]
print(label.head())

   toxic  severe_toxic  obscene  threat  insult  identity_hate
0      0             0        0       0       0              0
1      0             0        0       0       0              0
2      0             0        0       0       0              0
3      0             0        0       0       0              0
4      0             0        0       0       0              0


# Lower case the data and remove new lines and "#" "@"

In [ ]:
import nltk                               
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
import re                                  # library for regular expression operations
import string                              # for string operations
from nltk.corpus import stopwords 

In [ ]:
comment = comment.apply(lambda x: x.lower())

In [ ]:
comment.head()

0    explanation\nwhy the edits made under my usern...
1    d'aww! he matches this background colour i'm s...
2    hey man, i'm really not trying to edit war. it...
3    "\nmore\ni can't make any real suggestions on ...
4    you, sir, are my hero. any chance you remember...
Name: comment_text, dtype: object

In [ ]:
comment = comment.apply(lambda x: x.replace('\n', ' '))
comment = comment.apply(lambda x: x.replace('#', ' '))
comment = comment.apply(lambda x: x.replace('@', ' '))

In [ ]:
comment.head()

0    explanation why the edits made under my userna...
1    d'aww! he matches this background colour i'm s...
2    hey man, i'm really not trying to edit war. it...
3    " more i can't make any real suggestions on im...
4    you, sir, are my hero. any chance you remember...
Name: comment_text, dtype: object

# Slang Conversion

In [ ]:
#for slang conversion
CONTRACTION_MAP = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"I'd've": "I would have",
"I'll": "I will",
"I'll've": "I will have",
"I'm": "I am",
"I've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have",}

In [ ]:
SLANG={"AFAIK":"As Far As I Know",
"AFK":"Away From Keyboard",
"ASAP":"As Soon As Possible",
"ATK":"At The Keyboard",
"ATM":"At The Moment",
"A3":"Anytime, Anywhere, Anyplace",
"BAK":"Back At Keyboard",
"BBL":"Be Back Later",
"BBS":"Be Back Soon",
"BFN":"Bye For Now",
"B4N":"Bye For Now",
"BRB":"Be Right Back",
"BRT":"Be Right There",
"BTW":"By The Way",
"B4":"Before",
"B4N":"Bye For Now",
"CU":"See You",
"CUL8R":"See You Later",
"CYA":"See You",
"FAQ":"Frequently Asked Questions",
"FC":"Fingers Crossed",
"FWIW":"For What It's Worth",
"FYI":"For Your Information",
"GAL":"Get A Life",
"GG":"Good Game",
"GN":"Good Night",
"GMTA":"Great Minds Think Alike",
"GR8":"Great!",
"G9":"Genius",
"IC":"I See",
"ICQ":"I Seek you (also a chat program)",
"ILU": "I Love You",
"IMHO":"In My Honest/Humble Opinion",
"IMO":"In My Opinion",
"IOW":"In Other Words",
"IRL":"In Real Life",
"KISS":"Keep It Simple, Stupid",
"LDR":"Long Distance Relationship",
"LMAO":"Laugh My A.. Off",
"LOL":"Laughing Out Loud",
"LTNS":"Long Time No See",
"L8R":"Later",
"MTE":"My Thoughts Exactly",
"M8":"Mate",
"NRN":"No Reply Necessary",
"OIC":"Oh I See",
"PITA":"Pain In The A..",
"PRT":"Party",
"PRW":"Parents Are Watching",
"QPSA?":"Que Pasa?",
"ROFL":"Rolling On The Floor Laughing",
"ROFLOL":"Rolling On The Floor Laughing Out Loud",
"ROTFLMAO":"Rolling On The Floor Laughing My A.. Off",
"SK8":"Skate",
"STATS":"Your sex and age",
"ASL":"Age, Sex, Location",
"THX":"Thank You",
"TTFN":"Ta-Ta For Now!",
"TTYL":"Talk To You Later",
"U":"You",
"U2":"You Too",
"U4E":"Yours For Ever",
"WB":"Welcome Back",
"WTF":"What The F…",
"WTG":"Way To Go!",
"WUF":"Where Are You From?",
"W8":"Wait…",
"7K":"Sick:-D Laugher"}

In [ ]:
def slangs(x):
  y=x.split()
  for i in range(len(y)):
    if y[i] in SLANG:
      y[i]=SLANG[y[i]]
    if y[i] in CONTRACTION_MAP:
      y[i]=CONTRACTION_MAP[y[i]]  
  return ' '.join(y)    

In [ ]:
comment=comment.apply(lambda x: slangs(x))

In [ ]:
comment.head()

0    explanation why the edits made under my userna...
1    d'aww! he matches this background colour i am ...
2    hey man, i am really not trying to edit war. i...
3    " more i cannot make any real suggestions on i...
4    you, sir, are my hero. any chance you remember...
Name: comment_text, dtype: object

# Spell check

In [ ]:
!pip install spellchecker
!pip install pyspellchecker

In [ ]:
from spellchecker import SpellChecker

In [ ]:
def spell_checker(x):
  spell=SpellChecker()
  correct=[]
  misspelled=spell.unknown(x.split())
  for word in x.split():
    if word in misspelled:
      correct.append(spell.correction(word))
    else:
      correct.append(word)
  return ' '.join(correct)    

In [ ]:
comment=comment.apply(lambda x: spell_checker(x))

In [ ]:
comment.head()

#Remove punctutation symbols

In [ ]:
import nltk.tokenize
nltk.download('punkt')

In [ ]:
from nltk.tokenize import sent_tokenize,word_tokenize

In [ ]:
def remove_punct(x):
  exclude = set(string.punctuation)
  x=list(x)
  for a in range(len(x)):
    if x[a] in exclude:
        x[a]=" "
  x="".join(x)
  return x

In [ ]:
comment=comment.apply(lambda x: remove_punct(x))

In [ ]:
comment.head()

# Remove stop words

In [ ]:
def remove_stopwords(x):
  x=word_tokenize(x)
  stopwords_all = stopwords.words('english')
  x = " ".join(i for i in x if not(i in stopwords_all))
  return x


In [ ]:
comment=comment.apply(remove_stopwords)

In [ ]:
comment.head(10)

# Lematization

In [ ]:
nltk.download('wordnet')

In [ ]:
from nltk.stem import WordNetLemmatizer

In [ ]:
def lematization(x):
  x=word_tokenize(x)
  lemmatizer = WordNetLemmatizer()
  for w in range(len(x)):
    x[w]=lemmatizer.lemmatize(x[w])
  x=" ".join(x)
  return x


In [ ]:
comment=comment.apply(lematization)

In [ ]:
comment.head(10)

# Stemming

In [ ]:
from nltk.stem.snowball import SnowballStemmer


In [ ]:
def stemmer(x):
  stemmer = SnowballStemmer("english")
  x=word_tokenize(x) 
  x=" ".join(stemmer.stem(i) for i in x)
  return x  

In [ ]:
comment=comment.apply(stemmer)

In [ ]:
comment.head()

# TF-IDF Matrix

In [ ]:
import math

In [ ]:
def term_feq(w,i,comment):
  x=(comment.iloc[i]).split()
  count=0
  for a in x:
    if (a==w):
      count=count+1
  return count

In [ ]:
def idf(w,df):
  count=0
  for i in range(len(df)):
    x=(df.iloc[i]).split()
    if (w in x):
      count=count+1
  return (len(df)/count)

In [ ]:
def TF_IDF(df):
  word=[]
  ans=[]
  for i in range(len(df)): 
    x=(df.iloc[i]).split()
    word=word+x
  word=list(set(word))
  for i in range(len(df)): 
    x=[]
    for w in word:
      x.append(term_feq(w,i,df)*math.log(idf(w,df)))
    ans.append(x)
  return pd.DataFrame(ans,columns=word)

MAKING TF-IDF FOR FIRST 100 comments 

In [ ]:
tf_idf=TF_IDF(comment.head(100))

In [ ]:
tf_idf.head()

# Removal of frequent words

In [ ]:
from collections import Counter
cnt=Counter()
freq=Counter()
for text in comment.values:
  for word in text.split():
    cnt[word]+=1

freq=cnt.most_common(30) 
freq_words=[]
for i in freq:
  freq_words.append(i[0])

In [ ]:
def remove_freq(x):
  y=x.split()
  for i in y:
    if i in freq_words:
      y.remove(i)
  return ' '.join(y) 

In [ ]:
comment=comment.apply(lambda x: remove_freq(x))

In [ ]:
comment.head()

# POS Tagging


In [ ]:
nltk.download('averaged_perceptron_tagger')

In [ ]:
def pos(x):
  words=word_tokenize(x)
  ans=[]
  for word in words:
    ans.append((nltk.pos_tag([word])))
  print(ans)


In [ ]:
(comment.head()).apply(pos)